In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
cwt_path = "/Users/alfioleanza/progetto_tesi/dataset-eeg/miltiadous_deriv_uV_d1.0s_o0.0s/cwt"
labels_path = "/Users/alfioleanza/progetto_tesi/dataset-eeg/inference_20250327_171717/true_pred.csv"

data_split = {
    "train": [37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21],
    "val": [54, 55, 56, 57, 58, 59, 79, 80, 81, 82, 83, 22, 23, 24, 25, 26, 27, 28],
    "test": [60, 61, 62, 63, 64, 65, 84, 85, 86, 87, 88, 29, 30, 31, 32, 33, 34, 35, 36]
}

df_labels = pd.read_csv(labels_path)

In [3]:
df_labels['train_label'] = (df_labels['pred_label'] == df_labels['true_label']).astype(int)
df_labels['train_label'].unique()

array([1, 0])

In [4]:
df_labels

,crop_file,activation_values,dataset,softmax_values,pred_label,true_label,original_rec,crop_start_sample,crop_end_sample,train_label
0,00000848.npy,[ 1.0532185 -1.5921667 -0.8496607],training,[0.8195938 0.05817313 0.12223307],0,0,sub-037,0,499.0,1
1,00000849.npy,[ 0.8592405 -0.8918216 -0.32704002],training,[0.67615795 0.1173739 0.20646815],0,0,sub-037,500,999.0,1
2,00000850.npy,[ 0.89024836 -1.5663235 -0.599953 ],training,[0.7627442 0.06538879 0.17186707],0,0,sub-037,1000,1499.0,1
3,00000851.npy,[ 0.9642988 -1.5698762 -0.83838975],training,[0.80374086 0.06375847 0.13250075],0,0,sub-037,1500,1999.0,1
4,00000852.npy,[ 0.9176249 -1.4399661 -0.5990748],training,[0.76098704 0.07202587 0.1669871 ],0,0,sub-037,2000,2499.0,1
...,...,...,...,...,...,...,...,...,...,...
69789,00069084.npy,[-3.1664958 -1.7627978 -0.22231679],validation,[0.04155363 0.16913258 0.78931385],2,2,sub-026,446500,446999.0,1
69790,00069085.npy,[-1.5803745 -0.6570947 -0.06765006],validation,[0.12412163 0.3124804 0.563398 ],2,2,sub-026,447000,447499.0,1
69791,00069086.npy,[-0.6699866 -0.1551984 -0.40366274],validation,[0.2513547 0.4205878 0.3280575],1,2,sub-026,447500,447999.0,0
69792,00069087.npy,[-0.6042401 -0.30352032 -0.33559638],validation,[0.27329725 0.36917832 0.35752445],1,2,sub-026,448000,448499.0,0


In [5]:
class CWT_Dataset(Dataset):
    def __init__(self, file_list, labels, root_dir):
        self.file_list = file_list
        self.labels = labels
        self.root_dir = root_dir
    
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, idx):
        file_name = self.file_list[idx]
        label = self.labels[idx]
        file_path = os.path.join(self.root_dir, file_name)
        cwt_data = np.load(file_path)
        cwt_data = torch.tensor(cwt_data, dtype=torch.float32)
        cwt_data = cwt_data.permute(2, 0, 1)
        return cwt_data, label

def create_dataloader(split, batch_size=16):
    subset = df_labels[df_labels['original_rec'].isin([f"sub-{s:03d}" for s in data_split[split]])]
    file_list = list(subset["crop_file"])
    labels = list(subset["train_label"])
    dataset = CWT_Dataset(file_list, labels, cwt_path)
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)

class ResNet18_19Channels(nn.Module):
    def __init__(self, num_classes=2):
        super(ResNet18_19Channels, self).__init__()
        self.model = models.resnet18(pretrained=True)
        
        self.model.conv1 = nn.Conv2d(19, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)

    def forward(self, x):
        return self.model(x)

def train_model(model, train_loader, val_loader, epochs=10):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
        
        val_acc = evaluate_model(model, val_loader)
        print(f"Epoch {epoch+1}: Loss={running_loss/len(train_loader):.4f}, Train Acc={100.*correct/total:.2f}%, Val Acc={val_acc:.2f}%")

def evaluate_model(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    return 100. * correct / total

In [6]:
train_loader = create_dataloader("train")
val_loader = create_dataloader("val")
test_loader = create_dataloader("test")

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResNet18_19Channels(num_classes=2).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

/Users/alfioleanza/progetto_tesi/venv/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/alfioleanza/progetto_tesi/venv/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
train_model(model, train_loader, val_loader, epochs=10)

Epoch 1:  16%|█▋        | 414/2530 [13:17<1:07:53,  1.93s/it]


KeyboardInterrupt: 